In [7]:
import pandas as pd
import numpy as np
import random
import time

def create_all_budgets():
    """
    This single function runs the entire data pipeline to go from the
    initial input files to the final, comprehensive budget file.
    """
    start_time = time.time()
    
    # --- Step 1: Load and Merge Initial Data ---
    print("--- Step 1: Loading and Merging Initial Data ---")
    try:
        students_social_df = pd.read_csv("students_groups_social.csv")
        schedules_df = pd.read_csv("student_schedules_with_meals.csv")
    except FileNotFoundError as e:
        print(f"Error loading files: {e}. Please ensure source files are available.")
        return

    # Merge using the index of the left df and the 'StudentID' column of the right df
    df = pd.merge(students_social_df, schedules_df, on='StudentID', how='inner')
    print(f"Data loaded and merged in {time.time() - start_time:.2f} seconds.")

    # --- Step 2: Calculate Study Budget ---
    # This must be done on a weekly basis because lectures start in Week 2.
    print("\n--- Step 2: Calculating Study Budgets ---")
    
    time_slot_cols = [f'T_{i}' for i in range(672)]
    weeks_cols = [time_slot_cols[i*168:(i+1)*168] for i in range(4)]
    #
    df['StudyBudget'] = 10
    for w, week_cols in enumerate(weeks_cols, 1):
        # For Week 1, the study requirement is a full 40 hours as there are no lectures
        if w == 1:
            df['StudyBudget'] += np.random.normal(45, 4, size=len(df)).clip(25, 60).round().astype(int)
            continue

        # For Weeks 2, 3, and 4, calculate lectures and subtract from 40
        # Count all non-'Free' slots first
        fixed_slots = df[week_cols].apply(lambda r: (r != 'Free').sum(), axis=1)
        # Count 'Meal' slots specifically
        meal_slots = df[week_cols].apply(lambda r: (r == 'Canteen').sum(), axis=1)
        # The difference is the number of lecture slots
        lecture_slots = fixed_slots - meal_slots
        
        # The study budget for the week is what's left of the 40-hour work week
        # .clip(0) ensures the budget doesn't go below zero if lectures exceed 40 hours
        weekly_study_budget = (40 - lecture_slots).clip(0)
        
        # Add this week's study requirement to the total
        df['StudyBudget'] += weekly_study_budget
            
    print("Study budgets calculated.")

    # --- Step 3: Calculate Social Budget ---
    print("\n--- Step 3: Calculating Social Budgets ---")
    
    base_hours_wk1_yr1 = 30
    base_hours_wk1_other = 20
    base_hours_wk2_4_yr1 = 20
    base_hours_wk2_4_yr_other = 15
    
    # Calculate base budget over 4 weeks
    df['BaseSocialBudget'] = np.where(
        df['Year'] == 1,
        base_hours_wk1_yr1 + (base_hours_wk2_4_yr1 * 3),
        base_hours_wk1_other + (base_hours_wk2_4_yr_other * 3)
    )
    
    # Adjust for propensity
    df['PropensityScore'] = df['Social Propensity'] + df['Club Propensity']
    avg_propensity = df['PropensityScore'].mean()
    adjustment_factor = 1 + (((df['PropensityScore'] - avg_propensity) / avg_propensity) * 0.25)
    df['SocialBudget'] = (df['BaseSocialBudget'] * adjustment_factor).round().astype(int)
    
    print("Social budgets calculated.")
    
    # --- Step 4: Calculate Outdoor Budget ---
    print("\n--- Step 4: Calculating Outdoor Budgets ---")
    
    df_sorted = df.sort_values(by='SocialBudget', ascending=False)
    chunk_size = 50
    shuffled_indices = []
    for i in range(0, len(df_sorted), chunk_size):
        chunk = list(df_sorted.index[i:i + chunk_size])
        random.shuffle(chunk)
        shuffled_indices.extend(chunk)
    df_shuffled = df_sorted.reindex(shuffled_indices)

    n_students = len(df)
    weekly_hours = [1, 4, 12, 20]
    budgets_weekly = np.concatenate([
        np.full(int(n_students * 0.10), weekly_hours[0]),
        np.full(int(n_students * 0.40), weekly_hours[1]),
        np.full(int(n_students * 0.40), weekly_hours[2]),
        np.full(n_students - int(n_students*0.9), weekly_hours[3])
    ])
    
    df_shuffled['OutdoorBudget'] = budgets_weekly * 4
    df = pd.merge(df, df_shuffled[['OutdoorBudget']], left_index=True, right_index=True, suffixes=('', '_new'))
    
    # Clean up columns from the merge if necessary
    if 'OutdoorBudget_new' in df.columns:
        df['OutdoorBudget'] = df['OutdoorBudget_new']
        df.drop(columns=['OutdoorBudget_new'], inplace=True)

    print("Outdoor budgets calculated.")

    # --- Final Cleanup and Save ---
    
    final_budget_cols = ['StudyBudget', 'SocialBudget', 'OutdoorBudget']

    # Combine all columns from both input DataFrames (excluding duplicates), plus the new budget columns
    all_input_cols = students_social_df.columns.tolist() + [col for col in schedules_df.columns if col not in students_social_df.columns]
    output_cols = all_input_cols + final_budget_cols

    # Use reindex to keep all columns in the desired order
    final_df = df.reindex(columns=output_cols)

    output_filename = 'schedules_with_final_budgets2.csv'
    print(f"\n--- Saving final budgets to '{output_filename}' ---")
    final_df.to_csv(output_filename, index=False)
    
    total_duration = time.time() - start_time
    print(f"\nProcess complete. All budgets calculated and saved in {total_duration:.2f} seconds.")

# Run the entire budgeting process
create_all_budgets()

--- Step 1: Loading and Merging Initial Data ---
Data loaded and merged in 0.83 seconds.

--- Step 2: Calculating Study Budgets ---
Study budgets calculated.

--- Step 3: Calculating Social Budgets ---
Social budgets calculated.

--- Step 4: Calculating Outdoor Budgets ---
Outdoor budgets calculated.

--- Saving final budgets to 'schedules_with_final_budgets2.csv' ---

Process complete. All budgets calculated and saved in 7.45 seconds.
